# Importing Libraries

In [1]:
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
import tensorflow as tf

%matplotlib inline

# Feature Functions

In [2]:
# class Features:
""" Images should be sent in RGB format """
    
def resize(img,size):
    """size is a tuple"""
    """ returns resized images"""
    return cv2.resize(img,size)

def to_hsv(img):
    """return image in HSV space"""
    return cv2.cvtColor(img,cv2.COLOR_RGB2HSV)

def to_gray(img):
    """return image in gray space"""
    return cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

def sum(arr):
    """returns sum and no. of pixels between 20 and 240"""
    sum = 0
    count = 0
    for i in arr:
        for j in i:
            if(j>20 and j<240): #only pixels whose value is between 20 and 240
                sum+=j
                count+=1

    return (sum,count)

def pooling(image, pool_size, code, padding):
    """
    different codes for different pooling
    code min :min pooling
    code max :max pooling 
    code mean :mean pooling 
    code std :standard deviation pooling
    returns a image with padding operation and pooling operation
    """

    padded = arr = np.zeros((image.shape[0] + padding*2, 
                       image.shape[1] + padding*2))
    
    #  inserting image into zero array
    padded[int(padding):-int(padding), int(padding):-int(padding)] = image
    
    
    # print(f'original image size: {image.shape}')
    # print(f'padded image size: {padded.shape}')

    input_height, input_width = padded.shape
    pool_height, pool_width = pool_size

    # Calculate the output dimensions
    output_height = input_height - pool_height + 1
    output_width = input_width - pool_width + 1

    # Initialize the output data
    output_data = np.zeros((output_height, output_width))

    for i in range(output_height):
        for j in range(output_width):
            # Extract the region of interest (ROI)
            roi = padded[i : i + pool_height, j : j + pool_width]
            
            if code=='min':
                # Apply min pooling within the ROI
                output_data[i, j] = np.min(roi)

            if code=='max':
                # Apply max pooling within the ROI
                output_data[i, j] = np.max(roi)

            if code=='mean':
                # Apply mean pooling within the ROI
                output_data[i, j] = np.mean(roi)

            if code=='std':
                # Apply min pooling within the ROI
                output_data[i, j] = np.std(roi)


    # print(f'{code} pooled image size: {output_data.shape}')
    return output_data

def feature(data):
    """Return all the 12 features as a numpy array"""
    number,img,label = data
    img = resize(img,(250,250))

    #RGB SPACE
    r, g, b = cv2.split(img)
    sum_img = [sum(r),sum(g),sum(b),sum(r-g)]
    mean_features = [i[0]/i[1] for i in sum_img]
    mean_r,mean_g,mean_b,mean_rg = mean_features
    # 4 features done in RGB SPACE

    
    #HSV SPACE
    hsv = to_hsv(img)
    h,s,v = cv2.split(hsv)
    h = h/h.max()
    nH = np.count_nonzero(h>0.95)
    HHR = nH/h.size
    # HHR found

    
    #GRAY SPACE
    gray = to_gray(img)
    B_sum, B_size = sum(gray)
    B = B_sum/B_size # FOUND B

    #ENTROPY in gray space
    eq = cv2.equalizeHist(gray)
    unique, counts = np.unique(eq, return_counts=True)
    #only pixels whose value is between 20 and 240
    total_counts = counts[21:240].sum() 
    Ent = np.sum(np.array([-i*(i/total_counts)*math.log((i/total_counts),2) for i in counts[21:240]])) #Found Entropy

    #Calculating the 'G' features
    Ixy = gray
    min_Ixy = pooling(image=Ixy, pool_size=(3,3), code='min', padding=1)
    max_Ixy = pooling(image=Ixy, pool_size=(3,3), code='max', padding=1)
    mean_Ixy = pooling(image=Ixy, pool_size=(3,3), code='mean', padding=1)
    std_Ixy = pooling(image=Ixy, pool_size=(3,3), code='std', padding=1)
    
    g1 = Ixy - min_Ixy
    g2 = max_Ixy - Ixy
    g3 = Ixy - mean_Ixy
    g4 = std_Ixy
    g5 = Ixy
    
    G1 = g1.sum()/g1.size
    G2 = g2.sum()/g2.size
    G3 = g3.sum()/g3.size
    G4 = g4.sum()/g4.size
    G5 = g5.sum()/g5.size

    feature_all = [number,mean_r,mean_g,mean_b,mean_rg,HHR,Ent,B,G1,G2,G3,G4,G5,label]
    return feature_all

# Accessing Files

In [3]:
print(os.getcwd())
training_0_list = os.listdir('../India_95/training_data_0.7/non_anemic_0/')
training_1_list = os.listdir('../India_95/training_data_0.7/anemic_1/')
testing_0_list = os.listdir('../India_95/testing_data_0.3/non_anemic_0/')
testing_1_list = os.listdir('../India_95/testing_data_0.3/anemic_1/')

E:\Intelehealth\Code


In [4]:
training = []
testing = []
path = []
path.append('../India_95/training_data_0.7/non_anemic_0/')
path.append('../India_95/training_data_0.7/anemic_1/')
path.append('../India_95/testing_data_0.3/non_anemic_0/')
path.append( '../India_95/testing_data_0.3/anemic_1/')

data = [training_0_list,training_1_list,testing_0_list,testing_1_list]
for i in range(4):
    curr_data,curr_path = data[i], path[i]
    for j in curr_data:
        filepath = curr_path + j
        img = cv2.imread(filepath)        
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        label = i%2
        data_input = [j.split('.')[0],img,label]
        if i==0 or i==1:
            training.append(data_input)
        else:
            testing.append(data_input)

In [5]:
print('Length of Training Data',len(training))
print('Length of Testing Data',len(testing))

Length of Training Data 65
Length of Testing Data 30


# Generating Training Data

In [6]:
training_list = []

In [7]:
for i in training:
    training_list.append(feature(i))

print(len(training_list))

65


# Generating Testing Data

In [8]:
testing_list = []

In [9]:
for i in testing:
    testing_list.append(feature(i))

print(len(testing_list))

30


# Final CSV files

In [10]:
cols = ['number','mean_r','mean_g','mean_b','mean_rg','HHR','Ent','B','G1','G2','G3','G4','G5','label']

In [11]:
df_training = pd.DataFrame(training_list,columns=cols)
df_testing = pd.DataFrame(testing_list,columns=cols)

In [12]:
# df_training.to_csv('training.csv',index=False)
# df_testing.to_csv('testing.csv',index=False)